---
title: EpigenomeXcan test on Br rats
author: Sabrina Mi
date: 7/10/2024
description: Calculate associations between predicted gene expression in adipose tissue and BMI to identify significant gene, while I figure out high to scale up predicting epigenome step.
---

CPU times: 

1. Compute founders matrix (n_genes, 8)
    * 3190 genes: ~110s
2. Compute samples matrix (n_samples, n_genes, 8):
    * 3190 genes, 1 sample: ~4s
    * 3190 genes, 10 samples: ~40s

GPU times, combined steps 1 & 2:
1. Old code
    * 3190 genes, 340 samples: 32 minutes
2. New code removing inner foor loop
    * 1592 genes, 340 samples: 2 min 30 sec
    * 1575 genes, 340 samples: 2 min 5 sec
    * 19696 genes, 340 samples: 22 min 30 sec

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import h5py
import os
import time
import bisect
columns = ['ACI', 'BN', 'BUF', 'F344', 'M520', 'MR', 'WKY', 'WN']

2024-08-09 00:40:22.787012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 00:40:30.021105: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /soft/compilers/cudatoolkit/cuda-11.8.0/extras/CUPTI/lib64:/soft/compilers/cudatoolkit/cuda-11.8.0/lib64:/soft/libraries/trt/TensorRT-8.5.2.2.Linux.x86_64-gnu.cuda-11.8.cudnn8.6/lib:/soft/libraries/nccl/nccl_2.16.2-1+cuda11.8_x86_64/lib:/soft/libraries/cudnn/cudnn-11-linux-x64-v8.6.0.163/lib:/opt/cray/pe/gcc/11.2.0/snos/lib64:/opt/cray/pe/papi/6.0.0.14/lib64:/opt/cray/libfabric/1.11.0.4.125/lib64:/

In [2]:
gene_annot = pd.read_csv("/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/gene_mapping.txt")
genes_by_chrom = gene_annot.groupby('chromosome')

In [3]:
probabilities_dir = "/eagle/AIHPC4Edu/sabrina/Br_genotype_probabilities"
reference_dir = "/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/human"
output_dir = "/eagle/AIHPC4Edu/sabrina/Br_prediction_from_founders"

Realizing my haplotype probabilities storage is redundant, taking up extra space.

In [4]:
#| code-fold: true
probabilities_file = f'{probabilities_dir}/chr20_probabilities.h5'
with h5py.File(probabilities_file, 'a') as hf:
    for dataset_name in list(hf.keys()):
        if hf[dataset_name].shape[1] == 8:
            continue
        elif hf[dataset_name].shape[1] == 9:
            # Read the dataset
            data = hf[dataset_name][:]
            positions = data[:, 0]  # Assuming the positions column is the first column
            new_data = data[:, 1:]  # All columns except the first one
            
            # Create a temporary dataset without the positions column
            temp_dataset_name = f"temp_{dataset_name}"
            hf.create_dataset(temp_dataset_name, data=new_data)
            
            # Delete the original dataset
            del hf[dataset_name]
        
            # Rename the temporary dataset to the original dataset name
            hf.move(temp_dataset_name, dataset_name)
        
    hf.create_dataset('positions', data=positions)
        # Store the positions vector as metadata (attribute) for the dataset



split genes by chromosome number
for individual, query haplotype probabilities at each gene tss, so we return an 8 x n_gene matrix. by batching by genes n_genes should be in the 1-2K range. stack to return a 3d array with dimensions n_samples x 8 x n_gene
next query reference epigenome at each gene [446:450, CAGE_index], return 8 x n_gene matrix
matrix multiply with something like tf.transpose(tf.tensordot(_W, _X, axes=[[1],[1]]),[1,0,2])
https://stackoverflow.com/questions/41870228/understanding-tensordot

In [4]:
def reference_epigenome_matrix(chr, genes_df, track = 5278):
    reference_file = f'{reference_dir}/{chr}_genes.h5'
    with h5py.File(reference_file, 'r') as ref:
        rows = []
        for gene in genes_df['gene']:
            founder_predictions = ref[gene][:, 446:450, track]
            rows.append(founder_predictions)
        ref_matrix = np.stack(rows, axis=0)
        ref_tensor = tf.reduce_mean(tf.convert_to_tensor(ref_matrix, dtype=tf.float32), axis=2)
        return ref_tensor

In [5]:
def probabilities_matrix(chr, genes_df, individuals):
    probabilities_file = f'{probabilities_dir}/{chr}_probabilities.h5'
    with h5py.File(probabilities_file, 'r') as prob:
        positions = prob['positions'][:]
        population_prob = []
        indices = []
        for tss in genes_df['tss']:
            index = bisect.bisect_left(positions, tss)
            if index == 0:
                indices += [0,0]
            elif index == len(positions):
                indices += [index-1,index-1]
            else: # 0 < index < len(positions)
                indices += [index-1, index]
        for sample in individuals:
            dataset = prob[sample][:]
            sample_prob = tf.convert_to_tensor(dataset[indices], dtype = tf.float32)
            sample_prob = tf.reshape(sample_prob, (-1, 2, sample_prob.shape[1]))
            sample_prob = tf.reduce_mean(sample_prob, axis=1)
            population_prob.append(sample_prob)
        prob_tensor = tf.stack(population_prob, axis=0)
        return prob_tensor

In [6]:
def predict_epigenome(chr, genes_df, individuals, output_file, track):
    ref_tensor = reference_epigenome_matrix(chr, genes_df, track)
    prob_tensor = probabilities_matrix(chr, genes_df, individuals)
    epigenome_tensor = tf.einsum('ijk,jk->ij', prob_tensor, ref_tensor)
    epigenome_df = pd.DataFrame(epigenome_tensor.numpy(), columns=genes_df['gene'], index = individuals)
    epigenome_df.to_csv(output_file)


In [7]:
with open('/eagle/AIHPC4Edu/sabrina/Br_genotype_probabilities/individuals.txt', 'r') as f:
    individuals = f.read().splitlines()
#pheno = pd.read_csv('/home/s1mi/enformer_rat_data/phenotypes/pheno.fam', sep = '\t', index_col = 'IID')

### Example with CAGE Brain

In [9]:
for chr, genes_df in genes_by_chrom:
    tic = time.perf_counter()
    predict_epigenome(chr, genes_df, individuals, f'{output_dir}/{chr}_CAGE_brain__predict.txt', track = 4980)
    toc = time.perf_counter()
    print(f'{chr}, {len(genes_df)} genes:', toc - tic, 'seconds')

2024-08-09 00:45:57.658084: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /soft/compilers/cudatoolkit/cuda-11.8.0/extras/CUPTI/lib64:/soft/compilers/cudatoolkit/cuda-11.8.0/lib64:/soft/libraries/trt/TensorRT-8.5.2.2.Linux.x86_64-gnu.cuda-11.8.cudnn8.6/lib:/soft/libraries/nccl/nccl_2.16.2-1+cuda11.8_x86_64/lib:/soft/libraries/cudnn/cudnn-11-linux-x64-v8.6.0.163/lib:/opt/cray/pe/gcc/11.2.0/snos/lib64:/opt/cray/pe/papi/6.0.0.14/lib64:/opt/cray/libfabric/1.11.0.4.125/lib64:/dbhome/db2cat/sqllib/lib64:/dbhome/db2cat/sqllib/lib64/gskit:/dbhome/db2cat/sqllib/lib32:/soft/compilers/cudatoolkit/cuda-12.4.1/extras/CUPTI/lib64:/soft/compilers/cudatoolkit/cuda-12.4.1/lib64:/soft/libraries/trt/TensorRT-8.6.1.6.Linux.x86_64-gnu.cuda-12.0/lib:/soft/libraries/nccl/nccl_2.21.5-1+cuda12.4_x86_64/lib:/soft/libraries/cudnn/cudnn-c

chr1, 3190 genes: 129.08863715000916 seconds
chr10, 1844 genes: 64.15786012192257 seconds
chr11, 651 genes: 37.914293350069784 seconds
chr12, 728 genes: 38.18615038192365 seconds
chr13, 739 genes: 38.521823541028425 seconds
chr14, 794 genes: 44.71535247191787 seconds
chr15, 778 genes: 43.91769809497055 seconds
chr16, 701 genes: 42.883026172057725 seconds
chr17, 555 genes: 38.34623588703107 seconds
chr18, 581 genes: 39.45503612293396 seconds
chr19, 654 genes: 35.40245614293963 seconds
chr2, 1592 genes: 162.26185088197235 seconds
chr20, 702 genes: 40.760752159985714 seconds
chr3, 1615 genes: 165.32191750698257 seconds
chr4, 1575 genes: 162.78142886899877 seconds
chr5, 1503 genes: 60.48483659804333 seconds
chr6, 1033 genes: 50.48902957001701 seconds
chr7, 1370 genes: 55.30841292592231 seconds
chr8, 1289 genes: 53.40656710800249 seconds
chr9, 982 genes: 48.54825172596611 seconds


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/eagle/AIHPC4Edu/sabrina/Br_predictions/HS_founder_epigenomes/human/chrX_genes.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)